<a href="https://colab.research.google.com/github/srilalithaveerubhotla/Mixup_LabelSmoothing_TensorflowLowlevel/blob/master/GradientTape_Basics_and_Linearclassifier_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How Gradient tape works ???

In [ ]:
!pip install tensorflow==2.0.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
a = tf.random.normal(shape =(3,3))
b = tf.random.normal(shape =(3,3))

with tf.GradientTape() as tape:
  tape.watch(a)
  c = tf.sqrt(tf.sqrt(a)+tf.square(a)+b)
  c_a = tape.gradient(c,a)
  print(c_a)

# Linear regression using Tensorflow

In [ ]:
input_dim = 2
output_dim = 1
learning_rate = 0.01

# weight matrixx
w = tf.Variable(tf.random.uniform(shape=(input_dim, output_dim)))

# Bias
b = tf.Variable(tf.zeros(shape=(output_dim,)))

# Predictions function
def prediction(features):
  return tf.matmul(features,w)+b

# loss Function
def lossfn(labels, predictions):
  return tf.reduce_mean(tf.square(labels - predictions))

# Training function
def training(x, y):
    with tf.GradientTape() as tape:
      predictions = prediction(x)
      loss = lossfn(y, predictions)
    # Note that `tape.gradient` works with a list as well (w, b).
      dloss_dw, dloss_db = tape.gradient(loss, [w, b])
    w.assign_sub(learning_rate * dloss_dw)
    b.assign_sub(learning_rate * dloss_db)
    return loss

In [ ]:
# artificial data to test the linear model
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

# Prepare a dataset.
num_samples = 10000
negative_samples = np.random.multivariate_normal(
    mean=[0, 10], cov=[[1, 0.75],[0.75, 1]], size=num_samples)
positive_samples = np.random.multivariate_normal(
    mean=[10, 0], cov=[[1, 0.75],[0.75, 1]], size=num_samples)
features = np.vstack((negative_samples, positive_samples)).astype(np.float32)
labels = np.vstack((np.zeros((num_samples, 1), dtype='float32'),
                    np.ones((num_samples, 1), dtype='float32')))

plt.scatter(features[:, 0], features[:, 1], c=labels[:, 0])

In [ ]:
# Shuffle the data.
indices = np.random.permutation(len(features))
features = features[indices]

labels = labels[indices]

# Create a tf.data.Dataset object for easy batched iteration
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
dataset = dataset.shuffle(buffer_size=1024).batch(256)

for epoch in range(10):
  for step, (x, y) in enumerate(dataset):
    loss = training(x, y)
  print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))

In [ ]:
predictions = prediction(features)
plt.scatter(features[:, 0], features[:, 1], c=predictions[:, 0] > 0.5)

In [ ]:
import time

t0 = time.time()
for epoch in range(20):
  for step, (x, y) in enumerate(dataset):
    loss = training(x, y)
t_end = time.time() - t0
print('Time per epoch: %.3f s' % (t_end / 20,))


In [ ]:
@tf.function
def train_on_batch(x, y):
  with tf.GradientTape() as tape:
    predictions = compute_predictions(x)
    loss = compute_loss(y, predictions)
    dloss_dw, dloss_db = tape.gradient(loss, [w, b])
  w.assign_sub(learning_rate * dloss_dw)
  b.assign_sub(learning_rate * dloss_db)
  return loss

In [ ]:
t0 = time.time()
for epoch in range(20):
  for step, (x, y) in enumerate(dataset):
    loss = training(x, y)
t_end = time.time() - t0
print('Time per epoch: %.3f s' % (t_end / 20,))